In [1]:
%pip install tensorflow --user

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install scikeras

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install wheel
%pip install pipwin
%pip install numpy
%pip install pandas
%pip install shapely

Note: you may need to restart the kernel to use updated packages.
  Created wheel for pipwin: filename=pipwin-0.5.2-py2.py3-none-any.whl size=8782 sha256=86911ca5f605b01a9fd2fef37053292183d522248c114b932b5d24c6bcc11faf
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\f4\a8\76\372879a40d2d4dd7de23efd867ed37112687429b8d0dda4545
  Created wheel for pyjsparser: filename=pyjsparser-2.7.1-py3-none-any.whl size=26004 sha256=e1edc9300bdee7538e064892965cf7cbbffc80ce4ca27c62163ee47eb7b0fff3
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\d5\88\34\ccb5bb40eb3178a134eb293e6c363928c5bcfba0b91031db76
Successfully built pipwin pyjsparser
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%pipwin` not found.


In [3]:
%pip install fiona
%pip install geopandas

  Using cached GDAL-3.5.2.tar.gz (756 kB)
  Running setup.py clean for gdal
Failed to build gdal
    Running setup.py install for gdal: started
    Running setup.py install for gdal: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'D:\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-k_1ushbv\\gdal\\setup.py'"'"'; __file__='"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-k_1ushbv\\gdal\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\User\AppData\Local\Temp\pip-wheel-wcvdqi6n'
       cwd: C:\Users\User\AppData\Local\Temp\pip-install-k_1ushbv\gdal\
  Complete output (139 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\osgeo
  copying osgeo\gdal.py -> build\lib.win-amd64-3.8\osgeo
  copying osgeo\gdalconst.py -> build\lib.win-amd64-3.8\osgeo
  copying osgeo\gdalnumeric.py -> build\lib.win-amd64-3.8\osgeo
  copying 

    copying gdal-utils\osgeo_utils\samples\gdal_ls.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
    copying gdal-utils\osgeo_utils\samples\gdal_lut.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
    copying gdal-utils\osgeo_utils\samples\gdal_mkdir.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
    copying gdal-utils\osgeo_utils\samples\gdal_remove_towgs84.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
    copying gdal-utils\osgeo_utils\samples\gdal_rm.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
    copying gdal-utils\osgeo_utils\samples\gdal_rmdir.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
    copying gdal-utils\osgeo_utils\samples\gdal_vrtmerge.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
    copying gdal-utils\osgeo_utils\samples\gdal_zip.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
    copying gdal-utils\osgeo_utils\samples\get_soundg.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
    copying gdal-utils\osgeo_utils\samples\histrep.py -> build

  Using cached Fiona-1.8.21.tar.gz (1.0 MB)
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'D:\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-gt87k77y\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-gt87k77y\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\User\AppData\Local\Temp\pip-pip-egg-info-udsm7_xp'
         cwd: C:\Users\User\AppData\Local\Temp\pip-install-gt87k77y\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


  Using cached geopandas-0.11.1-py3-none-any.whl (1.0 MB)
  Using cached Fiona-1.8.21.tar.gz (1.0 MB)
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'D:\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-tt52cjxq\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-tt52cjxq\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\User\AppData\Local\Temp\pip-pip-egg-info-t1s_yaj8'
         cwd: C:\Users\User\AppData\Local\Temp\pip-install-tt52cjxq\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import geopandas as gpd
from sklearn.compose import ColumnTransformer

ModuleNotFoundError: No module named 'geopandas'

In [2]:
# load dataset
property_df = pd.read_csv("../data/curated/full_data.csv")
X = property_df
X2 = X[['SA2_NAME21']].groupby(by="SA2_NAME21")["SA2_NAME21"].count()
X2 = pd.DataFrame(X2)
X2.columns=["count"]
single_counts = X2[X2['count']==1].reset_index()['SA2_NAME21'].tolist() # can improve this later with bootstrapping
X = X[~X["SA2_NAME21"].isin(single_counts)]
X.head(1)



,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count,SA2_CODE_2021
0,https://www.domain.com.au/warrandyte-vic-3113-...,Warrandyte VIC 3113,875.0,5,3,3,House,mud brick magic,211021262,145.209992,-37.747366,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262.0


In [3]:

# sf stands for shape file
australia_sf = gpd.read_file("../data/raw/shapefiles/Statistical_area_level2/SA2_2021_AUST_GDA2020.shp")
australia_sf.head(3)
# seperate suburbs that are in Victoria
vic_sf = australia_sf[australia_sf['STE_NAME21'] == 'Victoria']

# drop the null location ID values
vic_sf.dropna(inplace=True)
vic_sf.head(1)
vic_sf.where(vic_sf["SA2_NAME21"] == "Seaford (Vic.)").dropna()

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
vic_centroid_sf = vic_sf
vic_centroid_sf['geometry'] = vic_sf['geometry'].to_crs('+proj=cea').centroid.to_crs(vic_sf['geometry'].crs)
cbd_loc = vic_centroid_sf.where(vic_centroid_sf['SA2_NAME21'] == 'Melbourne CBD - North').dropna()['geometry']
cbd_loc

In [ ]:
# function taken from https://stackoverflow.com/a/56661833

from math import radians, cos, sin, asin, sqrt, inf

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# this function converts two floats to a point that is recognised by Shapely
def point_to_dist(row):
    dist = haversine(row["geometry"].y, row["geometry"].x, cbd_loc.y, cbd_loc.x)
    return dist


In [4]:
X2 = X[['SA2_NAME21']].groupby(by="SA2_NAME21")["SA2_NAME21"].count()
X2 = pd.DataFrame(X2)
X2.columns=["count"]
X3 = vic_centroid_sf.join(X2, how='inner',on='SA2_NAME21')

X3['cbd_dist'] = X3.apply (lambda row: point_to_dist(row), axis=1)
X3.sort_values(by=['cbd_dist'], inplace=True)
X3

NameError: name 'vic_centroid_sf' is not defined

In [5]:
pruned_X3 = X3[['SA2_NAME21', "cbd_dist"]]
pruned_X3.set_index('SA2_NAME21', inplace=True)
pruned_X3
#pruned_X3.join(X, how='left', on="SA2_NAME21")

NameError: name 'X3' is not defined

In [6]:
X_dists = X.join(pruned_X3, how='left', on="SA2_NAME21")

NameError: name 'pruned_X3' is not defined

In [9]:
sl = 50
scaler = StandardScaler()
numeric_features = [ 
                  "Median_tot_prsnl_inc_weekly", 
                  "Median_rent_weekly",
                  "Median_tot_fam_inc_weekly", 
                  "Median_tot_hhd_inc_weekly", 
                  "total_population",
                  "Bed", 
                  "Bath", 
                  "Park",
                  "Offence Count",
                  "cbd_dist"]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_features)])

y = X_dists['cost_text'].tolist()
print("\tnum training instances: ", len(y))

	num training instances:  10860


In [10]:
data = X_dists.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", 
              "cost_text", "SA2_NAME21"], axis=1)
data.astype({ 
          "Median_tot_prsnl_inc_weekly":"float", 
          "Median_rent_weekly":"float",
          "Median_tot_fam_inc_weekly":"float", 
          "Median_tot_hhd_inc_weekly":"float", 
          "total_population": "int64",
          "duration_mins": "category", 
          "Bed":"int32", 
          "Bath":"int32", 
          "Park":"int32",
          "property_type":"category",
          #"SA2_NAME21": "category",
          "Offence Count": "int32",
          "cbd_dist": "float"}).dtypes
data.shape

(10860, 12)

In [11]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(100, input_shape=(12,), kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation="sigmoid"))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [12]:
estimator = KerasRegressor(model=baseline_model, epochs=200, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=30027)

results = cross_val_score(estimator, preprocessor.fit_transform(data), y, cv=kfold)
print(results)

2022-10-01 13:20:56.052124: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2022-10-01 13:20:56.052168: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: uwu
2022-10-01 13:20:56.052176: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: uwu
2022-10-01 13:20:56.052288: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.65.1
2022-10-01 13:20:56.052312: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 390.154.0
2022-10-01 13:20:56.052318: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 390.154.0 does not match DSO version 515.65.1 -- cannot find working devices in this configuration
2022-10-01 13:20:56.052616: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary 

[nan nan nan nan nan nan nan nan nan nan]


/home/sesshah/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/sesshah/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/sesshah/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py", line 762, in fit
    self._fit(
  File "/home/sesshah/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py", line 931, in _fit
    self._fit_keras_model(
  File "/home/sesshah/anaconda3/lib/python3.9/site